<h2>Lab 3: Least Squares</h2>

<b>Demo Date: </b> Oct. 13 <br>
<b>Due Date: </b> Oct. 15

In this lab you will implement two algorithms for solving an ill-conditioned least squares problem. We create an artificial and overdetermined least-squares problem by removing two columns of a $10 \times 10$ Hilbert matrix, a classic ill-conditioned matrix. As we remove the two columns from the matrix it is no longer a Hilbert matrix, but it creates an overdetermined system with a large condition number: $\approx 3,796,554,168$.

Implement an algorithm that you believe will compute the value of $x$ for the least squares problem $Ax \approx b$ as accurately as the problem allows. You will also implement an algorithm that you believe to produce inaccurate solutions due to the large condition number of the system. In this lab you do not need to worry about having a fast implementation, any non-vectorized implementation will be enough. 

Compare the two solutions in terms of the norm of their residual vector $Ax - b$. In the demo of your lab you should be able to explain your choices of algorithms and the results you obtained. Why did one method produce a solution with a smaller residual than the other? 

In [1]:
import numpy as np
import copy
import random
from scipy.linalg import hilbert
from scipy.optimize import lsq_linear

b = np.ones(10)
A = hilbert(10)
A = np.delete(A, 5, 1)
A = np.delete(A, 4, 1)

print('Condition number of A: ', np.linalg.norm(A) * np.linalg.norm(np.linalg.pinv(A)))

# Use this test instance while developing the algorithms. 
# This instance is much easier to debug than the ill-conditioned problem above. 
A_test = np.array([[1, 2, 2], [4, 4, 2], [4, 6, 4]], dtype=float)
b_test = np.array([3, 6, 10], dtype=float)



Condition number of A:  3796554172.5006285


In [2]:
def back_substituion(U, b):
    x = np.zeros((b.shape[0]))
    for j in range(U.shape[1]-1,-1,-1):
        # singular matrix
        if U[j][j] == 0:
            break 
        x[j] = b[j]/ U[j][j]
    
        for i in range(0,j):
            b[i] = b[i] - U[i][j] * x[j]
    return x.transpose()

In [7]:
def qr_MGS(A):
    Q = np.zeros((A.shape[0], A.shape[0]), dtype=float)
    R = np.zeros((A.shape[1], A.shape[1]), dtype=float)
    for k in range(0, A.shape[1]):
        #R[k, k] = np.linalg.norm(A[:, k])
        R[k, k] = np.sqrt(np.dot(A[:, k], A[:, k]))
        # stop if linear dependent
        if R[k, k] == 0:
            break
        #normalize current column
        Q[:, k] = A[:, k]/R[k, k]
        # substract from succeeding columns their components in current column
        for j in range(k + 1, A.shape[1]):
            R[k, j] = np.dot((Q[:,k].T),A[:, j])
            A[:, j] = A[:, j] - np.dot(R[k, j],Q[:, k])
    return Q,R
m, n = A.shape
Q, R = qr_MGS(copy.deepcopy(A))
c = Q.T.dot(b)
c1 = c[:n]
#c2 = c[m-n+1:]
#expected_r = np.linalg.norm(c2)

# find x using back substituion
x = back_substituion(R, c1)
print('x:\n',x)
r_ = np.linalg.norm(A.dot(x)-b)
print('r:\n',r_)
#print(expected_r)

x:
 [-4.13804873e+00  1.86609680e+02 -1.76040212e+03  4.79070453e+03
 -7.90389559e+04  2.12680538e+05 -2.08355829e+05  7.15781753e+04]
r:
 1.309788585467328e-05


In [4]:
def forward_substituion(L, b):
    x = np.zeros((b.shape[0]))
    for j in range(0, L.shape[1]):
        # singular matrix
        if L[j][j] == 0:
            break 
        x[j] = b[j]/ L[j][j]
    
        for i in range(j, L.shape[0]):
            b[i] = b[i] - L[i][j] * x[j]
    return x

In [5]:
def lu_factor(A):
    L = np.eye(A.shape[0])
    U = A.copy()
    for k in range(U.shape[1]):
        if U[k][k] == 0:
            break
        temp = U[k+1:,k]/U[k,k]
        L[k+1:,k] = temp
        # adding the dimention to increase the shape of the temp, to make temp*U[k] caculable
        # here is the citation for adding dimention：
        # https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html
        temp = np.expand_dims(temp, axis=1)
        U[k+1:] = U[k+1:] - np.multiply(temp,U[k])
        # print(U[k])
        # print(U[k+1:, k]/U[k, k] * U[k])
        # print(U[k+1:, k]/U[k, k])
    #print(U)
    #print(L)    
        
    return L, U

In [10]:
def lsq_Nor_Eq(A, b):
    #ATAx = ATb
    A1 = np.dot(A.T, A)
    b1 = np.dot(A.T, b)
    L, U = lu_factor(copy.deepcopy(A1))
    y = forward_substituion(L, copy.deepcopy(b1))
    #print(y)
    x = back_substituion(U, y)
    return x

x1 = lsq_Nor_Eq(copy.deepcopy(A), copy.deepcopy(b))
print('x:\n',x1)
r_1 = np.linalg.norm(A.dot(x)-b)
print('r:\n',r_1)

x:
 [ 4.35935965e+00 -1.42801682e+02  9.69400785e+02 -1.83950836e+03
  3.88340382e+03  6.10490621e+03 -1.99319729e+04  1.10156338e+04]
r:
 0.00013443774141366037
